<a href="https://colab.research.google.com/github/Daniil81704/Text-clusterization/blob/master/Text_clusterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installation-секция
!pip install -U sentence-transformers
!pip install hdbscan

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# import-секция
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import hdbscan

In [ ]:
sentences = ['Здравствуйте, я хочу заказать пиццу',
             'Добрый день, я бы хотел снять у вас одноместный номер на трое суток',
             'Естественная сладость яблока и моркови гармонично сочетается с солоноватостью сыра',
             'Сочные овощи станут основой теста, мука и яйцо помогут скрепить ингредиенты, а простые специи сбалансируют вкус',
             'Мы с семьей каждый год ходим в поход',
             'Отличная выдалась поездка на море в этом году!']

sentences_arr = np.array(sentences)

In [ ]:
# определение необходимых функций
def obtaining_embeddings(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

def get_clustered(labels):
  x = list(labels)
  res = []
  for i in range(0, max(labels)+1):
    lst_1 = np.where(x == np.int32(i))[0]
    for j in lst_1:
      res.append([i, sentences_arr[j], j])
  return res

In [ ]:
tokenizer = BertTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = BertModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
# получение эмбеддингов
embeddings = []
for i in range(len(sentences)):
  embeddings.append(obtaining_embeddings(sentences[i], model, tokenizer))
embeddings_arr = np.array(embeddings)

In [ ]:
# кластеризация предложений
clusterer = hdbscan.HDBSCAN(min_cluster_size = 2, min_samples=2, cluster_selection_epsilon=0.01, alpha=0.5)

# Обучение на данных
clusterer.fit(embeddings_arr)

# Получение меток кластеров
labels = clusterer.labels_

In [ ]:
res = get_clustered(labels)
print(labels)
print(res)

[1 1 2 2 0 0]
[[0, 'Мы с семьей каждый год ходим в поход', 4], [0, 'Отличная выдалась поездка на море в этом году!', 5], [1, 'Здравствуйте, я хочу заказать пиццу', 0], [1, 'Добрый день, я бы хотел снять у вас одноместный номер на трое суток', 1], [2, 'Естественная сладость яблока и моркови гармонично сочетается с солоноватостью сыра', 2], [2, 'Сочные овощи станут основой теста, мука и яйцо помогут скрепить ингредиенты, а простые специи сбалансируют вкус', 3]]
